<a href="https://colab.research.google.com/github/Danil1D/Machine-learning/blob/main/%D0%94%D0%B0%D1%88%D0%BA%D0%B5%D0%B2%D0%B8%D1%87_%D0%BB%D0%B0%D0%B1%E2%84%967(%D0%B7%D0%B0%D0%B2%D0%B4_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# 1. Завантаження датасету
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic, SVD

data = Dataset.load_builtin('ml-100k')

dataset = data.raw_ratings
print(dataset[:5])


[('196', '242', 3.0, '881250949'), ('186', '302', 3.0, '891717742'), ('22', '377', 1.0, '878887116'), ('244', '51', 2.0, '880606923'), ('166', '346', 1.0, '886397596')]


In [15]:
# 2. Виведення перших 5 рядків
dataset = data.raw_ratings
print(dataset[:5])

[('196', '242', 3.0, '881250949'), ('186', '302', 3.0, '891717742'), ('22', '377', 1.0, '878887116'), ('244', '51', 2.0, '880606923'), ('166', '346', 1.0, '886397596')]


In [16]:
# 3. Реалізація двох алгоритмів
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)

algo_knn = KNNBasic()
algo_knn.fit(trainset)
predictions_knn = algo_knn.test(testset)

algo_svd = SVD()
algo_svd.fit(trainset)
predictions_svd = algo_svd.test(testset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [17]:
# 4. Крос-валідація для підбору оптимальних параметрів
knn_results = cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)

svd_results = cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7780  0.7778  0.7732  0.7671  0.7692  0.7731  0.0044  
Fit time          0.40    0.57    0.43    0.41    0.57    0.48    0.08    
Test time         7.32    4.36    3.52    3.91    4.11    4.65    1.36    
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7387  0.7401  0.7384  0.7366  0.7359  0.7379  0.0015  
Fit time          1.55    1.60    1.59    2.14    1.82    1.74    0.22    
Test time

In [18]:
# 5. Вибір найкращого алгоритму
knn_mae = sum(knn_results['test_mae']) / len(knn_results['test_mae'])
svd_mae = sum(svd_results['test_mae']) / len(svd_results['test_mae'])

print(f"Середня MAE для KNN: {knn_mae}")
print(f"Середня MAE для SVD: {svd_mae}")

if knn_mae < svd_mae:
    print("KNN має кращий результат з меншою MAE.")
    best_algo = algo_knn
else:
    print("SVD має кращий результат з меншою MAE.")
    best_algo = algo_svd

Середня MAE для KNN: 0.7730610288384472
Середня MAE для SVD: 0.7379369739399786
SVD має кращий результат з меншою MAE.


In [19]:
# 6. Виведення рекомендацій для конкретного користувача
user_id = str(196)
items = data.build_full_trainset().all_items()
items = list(map(str, items))

predictions = [algo_svd.predict(user_id, item) for item in items]
top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

recommended_items = [pred.iid for pred in top_predictions]
print("Топ-10 фільмів для користувача з ID 196:", recommended_items)


Топ-10 фільмів для користувача з ID 196: ['127', '272', '64', '483', '197', '286', '178', '427', '357', '318']
